# Start Up: 
Before Running the cell below, you must ensure that these have been run in Terminal **IN ORDER** : 
- conda update -n base -c defaults conda 

    - cd SageMaker
    
      - cd yelp-dataset-challenge-1-ds
      
         - conda env create -f environment.yml
          
            - source activate ydc1 
                
                - pip install python-decouple
                  
                  - pip install pprintpp
                  
# Spacy Installs: 

   - python -m spacy download en_core_web_lg

        - python -m spacy link en_core_web_lg en

In [80]:
import json

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
#import s3
#from pprintpp import pprint as pp
from sklearn.externals import joblib
nlp = spacy.load('en')

# Load in Bucket
#bucket = s3.Bucket('yelpchallenge1')
# Look inside the bucket.
#bucket.contents

In [ ]:
                    ### ***** DO NOT RUN. ******* #### 
                  ### ***** ALREADY INSTALLED. ****** ###

# Only have to run this once.
# Installs the .csv 'Locally' on SageMaker Instance

#bucket.get('datasets/df.csv', 'df.csv')

# Installing user.json 'Locally'
#bucket.get('datasets/user.json', 'user.json')


    # Load in Bucket
# bucket = s3.Bucket('yelpchallenge1')
    # Look inside the bucket.
# bucket.contents

# Getting Started: Imports

In [4]:
    # Read-in final.csv
final_df = pd.read_csv('final_df.csv')

    # Read-in df.csv
#df = pd.read_csv('df.csv', nrows=250000)


    # Read-in review_df.csv
#review_df = pd.read_csv('review.csv')

    # Read-in dtm_final.csv (FINAL)
dtm_final = pd.read_csv('dtm_final.csv')

    # import Vectorizer Model
#vect2 = joblib.load('vect_2.sav')

In [5]:
dtm_final.head(5)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,-PRON-,-PRON-,-PRON- -PRON-,-PRON- a,-PRON- all,-PRON- also,-PRON- and,...,worth,would,would be,would have,would not,wrong,year,yelp,yet,stars
0,0,0,0,0.069829,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,1.0
1,1,1,1,0.343331,0.0,0.000000,0.0,0.0,0.0,0.047998,...,0.052035,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,5.0
2,2,2,2,0.314034,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.093163,0.0,0.000000,5.0
3,3,3,3,0.080641,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.087372,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,5.0
4,4,4,4,0.515313,0.0,0.020392,0.0,0.0,0.0,0.043038,...,0.000000,0.029004,0.0,0.0,0.02601,0.027762,0.021839,0.0,0.028812,1.0


In [6]:
dtm_final = dtm_final.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [7]:
dtm_final = dtm_final.drop(columns = ['stars'])

In [8]:
dtm_final.head(1)

,-PRON-,-PRON-,-PRON- -PRON-,-PRON- a,-PRON- all,-PRON- also,-PRON- and,-PRON- be,-PRON- but,-PRON- can,...,work,worth,would,would be,would have,would not,wrong,year,yelp,yet
0,0.069829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
dtm_final.shape

(95000, 773)

# Model Prep: 

**Description**: 

Combining based on their *Unique Account ID's.*
The end Product will be One DataFrame Consisting of Each Account:
- **Name**, 
- **User_ID**,
- **Review_ID**,
- **Text**,
- **That Users respective review(s)**,
- **Interactions that Review (i.e: Cool, Funny, Useful)**  

The goal of the model is to have the ability to type in the Review you are wanting to post on Yelp, and give the User the ability to Predict What type of Interaction they would potentially receive and Total Number of each interaction. The model Accuracy will be Displayed beside the Prediction. 

In [10]:
# Imports: 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
# Creating Variables for Training 
interactions1 = final_df['useful']

In [31]:
# Splitting Dataset into training and test set

    # Using 10,000 Rows. 

reviews = final_df['text'].iloc[0:10000]
interaction = interactions1.iloc[0:10000]

In [22]:
# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [23]:
# Defining
# tokenizer
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

In [42]:
# Creating Fake Review for Testing: 
fake_review = """
After finshing our dinner we heard a loud crash come from the back of the place The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation given that I was completley drunk and could not stop laughing. The beer was a little flat, the 'chicken' was a little gamey. 
Service was great 4 Stars.
"""

In [43]:
# tokenizer
vect_review = tfidf.fit_transform(tokenize(fake_review))

In [32]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()


# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])


#Tuning
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs= -1, verbose=1)
grid_search.fit(reviews, interaction)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/Users/haden/anaconda3/envs/ydc1/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  3.8min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [33]:
grid_search.best_score_

0.5387

In [34]:
grid_search.best_params_

{'clf__max_depth': 15,
 'clf__n_estimators': 10,
 'vect__max_df': 1.0,
 'vect__max_features': 500,
 'vect__min_df': 0.02}

In [44]:
# Fake Review Interaction 'Useful' Prediction: 

useful_pred = grid_search.predict([r'vect_review'])
print(useful_pred)

[0.]


# NEW Model Prep: As of 1/08/2019

In [11]:
# Dropping Unwanted Columns
#final_df = final.drop(columns=['Unnamed: 0', 'user_id', 'review_id', 'date'])
# Final_df.csv File was too Large. Cutting Down to First 95,000 Rows.
final_df = final_df.iloc[0:95000]

# Save Cleaned and Trimmed final_df.csv 
final_df.to_csv(index=True)
final_df.to_csv(r'final_df.csv')

In [12]:
final_df.head(5)

,Unnamed: 0,Unnamed: 0.1,useful,funny,cool,text,tokens
0,0,0,6.0,1.0,0,Total bill for this horrible service? Over $8G...,"['total', 'horrible', 'service', 'crooks', 'ac..."
1,1,1,0.0,0.0,0,I *adore* Travis at the Hard Rock's new Kelly ...,"['adore', 'travis', 'hard', 'rock', 'kelly', '..."
2,2,2,3.0,0.0,0,I have to say that this office really has it t...,"['office', 'organized', 'friendly', 'phillipp'..."
3,3,3,0.0,0.0,0,Went in for a lunch. Steak sandwich was delici...,"['went', 'lunch', 'steak', 'sandwich', 'delici..."
4,4,4,7.0,0.0,0,Today was my second out of three sessions I ha...,"['today', 'second', 'sessions', 'paid', 'sessi..."


In [13]:
final_df = final_df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [14]:
dtm = dtm_final

In [15]:
dtm.shape

(95000, 773)

In [12]:
dtm_final = dtm.iloc[0:95000]
# Save Trimmed dtm_final.csv 
dtm_final.to_csv(index=True)
dtm_final.to_csv(r'dtm_final.csv')

In [20]:
dtm_final.shape

(95000, 773)

# Testing Random Forest Regression Model: 

In [16]:
# Imports:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np 
import matplotlib.pyplot as plt
# import Vectorizer Model
#vect = joblib.load('vect_2.sav')

In [17]:
interactions = final_df[['useful', 'funny', 'cool']]

In [23]:
# Splitting Dataset into training and test set
X = dtm_final
y = interactions

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 0)

In [ ]:
# Fitting Random Forest Regression to Dataset
regressor = RandomForestRegressor(n_estimators = 15, random_state = 0)
regressor.fit(X_train, y_train)

In [27]:
#Pickling Model with JobLib
from sklearn.externals import joblib
joblib.dump(regressor, 'regressor.joblib')

['regressor.joblib']

In [18]:
# Import Model 
regressor = joblib.load('regressor.joblib')

In [19]:
# Creating Fake Review for Testing: 
fake_review = """
After finshing our dinner, we heard a loud crash come from the back of the place. The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation, given that I was completley drunk and could not stop laughing. The beer was a little flat, the chicken was a little gamey. 
Service was great though. four Stars.
"""

In [ ]:
interaction_pred = regressor.predict(fake_review)
print(fake_review)

# Testing Different Regressor: 

In [28]:
# Imports:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np 
import matplotlib.pyplot as plt

In [33]:
# Only Using ONE Interaction: 
interactions2 = final_df['useful']

# Splitting Dataset into training and test set
X = dtm_final
y = interactions2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 0)

In [ ]:
# Fitting Random Forest Regression to Dataset
regressor2 = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor2.fit(X_train, y_train)

In [ ]:
# Creating Fake Review for Testing: 
fake_review = ["""
After finshing our dinner, we heard a loud crash come from the back of the place. The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation, given that I was completley drunk and could not stop laughing. The beer was a little flat, the chicken was a little gamey. 
Service was great though. 4 Stars.
"""]